<a href="https://colab.research.google.com/github/GulnazaS/Header-s-generation/blob/main/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%B7%D0%B0%D0%B3%D0%BE%D0%BB%D0%BE%D0%B2%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import kagglehub
import pandas as pd

from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

In [2]:
!pip install transformers

# Preprocessing

In [3]:
# Download latest version
path = kagglehub.dataset_download("yutkin/corpus-of-russian-news-articles-from-lenta")

print("Path to dataset files:", path)

100%|██████████| 584M/584M [00:16<00:00, 37.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/versions/2


In [4]:
df = pd.read_csv('/root/.cache/kagglehub/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/versions/2/lenta-ru-news.csv')

<ipython-input-4-62e407b24f9b>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/root/.cache/kagglehub/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/versions/2/lenta-ru-news.csv')


In [5]:
df

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
...,...,...,...,...,...,...
800970,https://lenta.ru/news/2019/12/14/shnur/,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,NaN,ТВ и радио,2019/12/14
800971,https://lenta.ru/news/2019/12/14/dolg/,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,NaN,Все,2019/12/14
800972,https://lenta.ru/news/2019/12/14/dark_euro/,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,NaN,Политика,2019/12/14
800973,https://lenta.ru/news/2019/12/14/meteo/,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,NaN,Общество,2019/12/14


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800975 entries, 0 to 800974
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     800975 non-null  object
 1   title   800975 non-null  object
 2   text    800970 non-null  object
 3   topic   738973 non-null  object
 4   tags    773756 non-null  object
 5   date    800975 non-null  object
dtypes: object(6)
memory usage: 36.7+ MB


In [7]:
df.nunique()

,0
url,800964
title,797832
text,800037
topic,23
tags,94
date,7393


In [8]:
df.drop_duplicates()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
...,...,...,...,...,...,...
800970,https://lenta.ru/news/2019/12/14/shnur/,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,NaN,ТВ и радио,2019/12/14
800971,https://lenta.ru/news/2019/12/14/dolg/,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,NaN,Все,2019/12/14
800972,https://lenta.ru/news/2019/12/14/dark_euro/,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,NaN,Политика,2019/12/14
800973,https://lenta.ru/news/2019/12/14/meteo/,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,NaN,Общество,2019/12/14


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800975 entries, 0 to 800974
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     800975 non-null  object
 1   title   800975 non-null  object
 2   text    800970 non-null  object
 3   topic   738973 non-null  object
 4   tags    773756 non-null  object
 5   date    800975 non-null  object
dtypes: object(6)
memory usage: 36.7+ MB


In [10]:
df.describe()

,url,title,text,topic,tags,date
count,800975,800975,800970,738973,773756,800975
unique,800964,797832,800037,23,94,7393
top,https://lenta.ru/news/2004/06/21/hostage/,В Москве объявлено штормовое предупреждение,"РИА ""Новости""",Россия,Все,2019/12/05
freq,2,21,291,160445,453762,284


In [11]:
del df['url'], df['topic'], df['tags'], df['date']

In [12]:
df

,title,text
0,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...
1,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про..."
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
3,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...
4,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю..."
...,...,...
800970,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...
800971,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...
800972,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...
800973,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...


In [13]:
df.drop_duplicates()

,title,text
0,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...
1,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про..."
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
3,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...
4,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю..."
...,...,...
800970,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...
800971,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...
800972,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...
800973,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800975 entries, 0 to 800974
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   800975 non-null  object
 1   text    800970 non-null  object
dtypes: object(2)
memory usage: 12.2+ MB


In [15]:
df.isnull().sum()

,0
title,0
text,5


In [16]:
df.dropna()

,title,text
0,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...
1,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про..."
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
3,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...
4,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю..."
...,...,...
800970,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...
800971,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...
800972,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...
800973,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...


In [17]:
df.isnull().sum()

,0
title,0
text,5


In [18]:
df[df.isna().any(axis=1)]

,title,text
150785,Прокуратура не нашла национальной почвы в напа...,NaN
153075,При штурме дома в Кизилюрте ранены шестеро,NaN
153931,В Стамбуле горит багажное отделение аэропорта ...,NaN
154591,РФ вышла из российско-украинского проекта Ан-70,NaN
582527,Гребенщиков заработал 10 тысяч рублей в омском...,NaN


In [19]:
df.dropna(inplace=True)

In [20]:
df[df.isna().any(axis=1)]

,title,text


# Model's load

In [21]:
DEVICE = torch.device("cuda:0")

# Russian language's GPT by SberDevices
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(DEVICE)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

In [22]:
df.dropna(inplace=True)

# Save DataFrame in CSV
output_path = '/content/df.csv'  # Folder's path, where dataset is stored
df.to_csv(output_path, index=False)


df.head(5000).to_csv(output_path, index=False)

In [23]:
# Path to .txt
train_path = '/content/df.csv'
# Dataset's creating
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)

# Dataloader's creating
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [24]:
training_args = TrainingArguments(
    output_dir="./finetuned",       # directory with output's data
    overwrite_output_dir=True,      # overwrite the contents of the output directory on each run
    num_train_epochs=3,            # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluate
    warmup_steps=10,                # number of steps for "warm-up" (control of learning speed)
    gradient_accumulation_steps=16, # gradient accumulation (16 steps of gradient accumulation for batch_size, emulation of calculation on a batch of 16 * 32 for weak GPUs)
    )

# Model's training

In [25]:
# Class of training
trainer = Trainer(
    model=model,
    args=training_args,          # parameters
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)
)

In [26]:
torch.cuda.empty_cache()

In [27]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gulnazsahapova1998 (gulnazsahapova1998-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=123, training_loss=3.1624803000349337, metrics={'train_runtime': 2815.2493, 'train_samples_per_second': 22.685, 'train_steps_per_second': 0.044, 'total_flos': 7370168711774208.0, 'train_loss': 3.1624803000349337, 'epoch': 2.984984984984985})

# Inference

In [40]:
text = "Сегодня президент Российской Федерации Борис Ельцин встретился в Кремле с директором ФСБ Николаем"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
print()

In [41]:
out = model.generate(input_ids,
                    do_sample=True,
                    num_beams=3,
                    temperature=1.1,
                    top_k=20,
                    top_p=0.8,
                    max_length=20,
                    )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Сегодня президент Российской Федерации Борис Ельцин встретился в Кремле с директором ФСБ Николаем Патрушевым. В


In [42]:
# Пример вероятностного сэмплирования
out = model.generate(input_ids, do_sample=True, temperature=1.3, max_length=20)

# Декодирование токенов
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Сегодня президент Российской Федерации Борис Ельцин встретился в Кремле с директором ФСБ Николаем Патрушевым и министром


In [47]:
text= 'Вряд ли перед олимпийским сезоном-2017/18 в мире фигурного катания был хоть один человек, который бы сомневался в том, что Медведева — будущая чемпионка Игр. До нее в женском одиночном катании давно никто так не доминировал. Последнее поражение Жени на тот момент датировалось ноябрем 2015-го. '
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
print()

In [45]:
text= 'Правительство России поручило администрации Краснодарского края заменить весь загрязнённый песок на побережье Анапы.'
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
print()

In [46]:
out = model.generate(input_ids,
                    do_sample=True,
                    num_beams=5,
                    temperature=1.1,
                    top_k=30,
                    top_p=0.8,
                    max_length=40,
                    )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Правительство России поручило администрации Краснодарского края заменить весь загрязнённый песок на побережье Анапы. Об этом сообщает РИА ""Новости"" со ссылкой на пресс-службу администрации Краснодарского края. Как сообщили в пресс
